In [17]:
import os
import matplotlib.pyplot as plt
import pandas
import numpy
import matplotlib
import pandas as pd
from matplotlib.axis import Axis
from matplotlib.figure import Figure
from matplotlib.patches import Rectangle

from env import *
from visualiser import *
from main import *
from constants import *
# import xlsxwriter
import csv
%matplotlib inline

In [18]:
SENSOR_RANGE = 10

n=range(2, 11)
ENV_SIZE=500
XL=250
YL=75

GOAL_X=250
GOAL_Y=425
MAX_T=20000

In [19]:
def calcDF(n):
    return [(SENSOR_RANGE+ROBOT_RADIUS)*(i-(n-1)/2) for i in range(n)] * n, [(SENSOR_RANGE+ROBOT_RADIUS)*(i-(n-1)/2) for i in range(n)for _ in range(n) ]

In [20]:
def plot_trajectory(env, fig:Figure = None, ax:Axis=None):

	if fig is None and ax is None:
		fig=matplotlib.pyplot.figure(figsize=(16, 16))
	if ax is None:
		ax=fig.add_subplot()
	# ax.invert_yaxis()

	posei=env.pose_history[:, env.N, :]
	ax.plot(posei[:env.t-1,0], posei[:env.t-1, 1], linestyle='dashdot',  label='Virtual Leader', color='blue', zorder=3)
	line=[None]*env.N
	for i in range(env.N):
					posei=env.pose_history[:, i, :]
					deadi=env.dead_history[:, i]
					posei=posei[~deadi]
					ax.plot(posei[:,0], posei[:, 1], label='Agent '+str(i), color='aqua')
	for wall in env.walls:
					ax.add_patch(Rectangle((wall.from_x, wall.from_y), wall.length_x, wall.length_y, color='black'))
	ax.scatter(env.xL0, env.yL0, marker='*', zorder=3, color='blue', s=150)
	ax.scatter(env.xG, env.yG, marker='*', zorder=3, color='blue', s=150)
	for agent in env.agents:
					ax.scatter(env.xL0+agent.dx, env.yL0+agent.dy, color='limegreen')
					ax.scatter(env.pose_history[env.t-1, agent.id, 0], env.pose_history[env.t-1, agent.id, 1], color='red' if agent.is_dead else 'limegreen')
	return fig


In [21]:
directory = '../run_exp_1'
if not os.path.exists(directory):
	os.makedirs(directory)
df=pandas.DataFrame(columns=['id','sensor_range' , 'n1', 'n2', 'n3', 'N', 'lived_agents', 'leader_goal_distance', 't', 'w1', 'w2', 'w3' ])
df.set_index('id', inplace=True)

In [22]:
# file_name=os.path.join(directory, 'var.npz')
n_max=50

csv_file_name=os.path.join(directory, 'report.csv')
if os.path.exists(csv_file_name):
	df=pandas.read_csv(csv_file_name, header=None, names=['id','sensor_range' , 'n1', 'n2', 'n3', 'N', 'lived_agents', 'leader_goal_distance', 't', 'w1', 'w2', 'w3'])
	run, _, n01, n02, n03, n00, _, _, _, _, _, _=list(df.iloc[-1])
	# cur_state=numpy.load(file_name)
	n03=int(n03)-1
	n00=int(n00)
	n01=int(n01)
	n02=int(n02)
	df.set_index('id', inplace=True)
else:
    # killed_list=[]
    n01=n_max
    n02 = n_max
    n03 = n_max
    run=0
    n00=2

fig=plt.Figure()
ax=fig.add_subplot()


In [26]:
n=3
DX,DY=calcDF(n)
env = Env(
        width=ENV_SIZE, height=ENV_SIZE, goal_x=GOAL_X, goal_y=GOAL_Y, N=n ** 2,
        desired_X=DX, desired_Y=DY, sensor_range=SENSOR_RANGE, leader_x=XL, leader_y=YL, robot_radius=ROBOT_RADIUS,
        sensor_detection_count=SENSOR_DETECTION_COUNT, buffer_size=MAX_T
    )
env.addObstacle(200, 200, 300, 300)
episode_gui(env, 1, 1, 1)

In [24]:
for n in range(n00, 11):
	print('n=', n)
	DX,DY=calcDF(n)
	env = Env(
        width=ENV_SIZE, height=ENV_SIZE, goal_x=GOAL_X, goal_y=GOAL_Y, N=n ** 2,
        desired_X=DX, desired_Y=DY, sensor_range=SENSOR_RANGE, leader_x=XL, leader_y=YL, robot_radius=ROBOT_RADIUS,
        sensor_detection_count=SENSOR_DETECTION_COUNT, buffer_size=MAX_T
    )
	env.addObstacle(200, 200, 300, 300)
	for n1 in range(n01, -1, -1):
		w1=n1/10
		print('w1=',w1)
		for n2 in range(n02,-1, -1):
			w2=n2/10
			print('w2=',w2)
			for n3 in range(n03, -1, -1):
				run=run+1
				w3=n3/10
				print('w3=',w3)
				print('run='+str(run))
				ep_file=os.path.join(directory, str(run)+'_'+str(n)+'_'+str(w1)+'_'+str(w2)+'_'+str(w3))
				#'id','sensor_range' , 'w1', 'w2', 'w3', 'N', 'lived_agents', 'leader_goal_distance', 't', 'v_mean'
				env.episode(w1, w2, w3)
				# env.save_episode(ep_file)

				row=[SENSOR_RANGE, n1, n2, n3, n, alive_agent_count(env), leader_goal_distance(env), env.t, w1, w2, w3]
				df.loc[run]=row
				print(row)
						# env.episode(w1/10, w2/10, w3/10, killed_list)
						# env.save_episode(ep_file+'a')

				# numpy.savez(file_name, run=run, n1=n1, n2=n2, n3=n3, n=n)

				plot_trajectory(env, ax=ax)
				plt_file=ep_file+'.jpg'
				fig.savefig(plt_file)
				ax.clear()
				# df.iloc[-1].to_csv(csv, mode='a', index=False, header=False)
				# run=run+1
				with open(csv_file_name, 'a',newline='' ) as f:
					writer=csv.writer(f)
					writer.writerow([run]+row)

				# run=run+1
				# df.to_csv()

			n03=n_max
		n02=n_max
	n01=n_max


n= 2
w1= 5.0
w2= 2.4
w3= 2.8
run=1349.0
[10, 50, 24, 28, 2, 0, 221.58723811973326, 53, 5.0, 2.4, 2.8]
w3= 2.7
run=1350.0
[10, 50, 24, 27, 2, 1, 0.0, 157, 5.0, 2.4, 2.7]
w3= 2.6
run=1351.0
[10, 50, 24, 26, 2, 1, 0.0, 162, 5.0, 2.4, 2.6]
w3= 2.5
run=1352.0
[10, 50, 24, 25, 2, 1, 0.0, 168, 5.0, 2.4, 2.5]
w3= 2.4
run=1353.0
[10, 50, 24, 24, 2, 1, 0.0, 173, 5.0, 2.4, 2.4]
w3= 2.3
run=1354.0
[10, 50, 24, 23, 2, 2, 0.0, 181, 5.0, 2.4, 2.3]
w3= 2.2
run=1355.0
[10, 50, 24, 22, 2, 1, 0.0, 187, 5.0, 2.4, 2.2]
w3= 2.1
run=1356.0
[10, 50, 24, 21, 2, 1, 0.0, 197, 5.0, 2.4, 2.1]
w3= 2.0
run=1357.0
[10, 50, 24, 20, 2, 2, 0.0, 206, 5.0, 2.4, 2.0]
w3= 1.9
run=1358.0
[10, 50, 24, 19, 2, 2, 0.0, 218, 5.0, 2.4, 1.9]
w3= 1.8
run=1359.0
[10, 50, 24, 18, 2, 2, 0.0, 229, 5.0, 2.4, 1.8]
w3= 1.7
run=1360.0
[10, 50, 24, 17, 2, 1, 0.0, 240, 5.0, 2.4, 1.7]
w3= 1.6
run=1361.0
[10, 50, 24, 16, 2, 2, 0.0, 253, 5.0, 2.4, 1.6]
w3= 1.5
run=1362.0


KeyboardInterrupt: 

In [ ]:
df1=df[df.leader_goal_distance==0]
df1=df1[df1.N==4]
df1[df1.N**2==df1.lived_agents]

In [ ]:
df1.count()

In [ ]:
df=pandas.read_csv(csv_file_name, header=None, names=['id','sensor_range' , 'w1', 'w2', 'w3', 'N', 'lived_agents', 'leader_goal_distance', 't' ])
id, SENSOR_RANGE, n01, n02, n03, n00, _, _, _, _=list(df.iloc[-1])
# list(df.iloc[-1])
# df.iloc[-1]
id